In [4]:
# Import necessary libraries
!pip install patool==1.12
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from google.colab import drive
import patoolib

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory containing the dataset
data_dir = '/content/extracted_images/leapGestRecog'  # Adjust the path if needed

# Use ImageDataGenerator for real-time data augmentation
datagen = ImageDataGenerator(validation_split=0.3, rescale=1.0/255.0)

# Load training and validation datasets using flow_from_directory
train_dataset = datagen.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),  # Resize images to 224x224 pixels (VGG16 input size)
    batch_size=16,  # Use batch size of 16
    class_mode='categorical',  # Use categorical labels (one-hot encoded)
    subset='training',
    seed=123  # Seed for reproducibility
)

validation_dataset = datagen.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),  # Resize images to 224x224 pixels (VGG16 input size)
    batch_size=16,  # Use batch size of 16
    class_mode='categorical',  # Use categorical labels (one-hot encoded)
    subset='validation',
    seed=123  # Seed for reproducibility
)

# Display class names to verify labels
class_names = list(train_dataset.class_indices.keys())
print("Class Names:", class_names)
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")

# Use `next()` to get a batch of data from train_dataset
images, labels = next(train_dataset)
print(f"Batch of images shape: {images.shape}")
print(f"Batch of labels shape: {labels.shape}")

# Load the pre-trained VGG16 model with custom layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the layers in the base model

# Add custom layers on top of VGG16
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # Output layer for the number of classes
])

# Compile the model with Adam optimizer
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to confirm the shapes
model.summary()

# Train the VGG16 model with EarlyStopping and Learning Rate Scheduler
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_dataset, verbose=2)
print(f"Validation accuracy: {val_acc:.4f}")

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy for VGG16')
plt.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 14000 images belonging to 10 classes.
Found 6000 images belonging to 10 classes.


AttributeError: 'DirectoryIterator' object has no attribute 'take'